In [1]:
import re, os, time, argparse
from pathlib import Path
import faiss
import numpy as np
import json
from tqdm import tqdm
from typing import List, Dict

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


MAX_TOKENS = 512

#Embedding model
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-base")
model = SentenceTransformer("intfloat/multilingual-e5-base")
#LLM model
llm_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
llm_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")


/home/jovyan/embeddings/embeddings/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [02:00<00:00, 30.22s/it]


## Limpieza del documento Mark Down

In [3]:
def md_to_latex(md_text):

    def convert_heading(match):
        title_text = match.group(2).strip()
        # Match numbers at the start of the heading (including multiple dots like "0.1.1")
        number_match = re.match(r'^([\d.]+)\s+', title_text)
        if number_match:
            number = number_match.group(1)  # Extract matched number
            title_text = title_text[len(number_match.group(0)):]  # Remove number + space
            if '.' in number:  # If it contains a dot (1.2 or 0.1.1) -> subsection = ##
                return rf' ## {title_text}'
            else:  # Otherwise (integer like "1", "2") -> section = #
                return rf' # {title_text}'
        return rf' # {title_text}'  # Default case

    md_text = re.sub(r'^(#)\s+(.+)', convert_heading, md_text, flags=re.MULTILINE)   
    md_text = re.sub(r'\s+}', '}', md_text)

    
    # regex pattern for removin figures
    pattern = r'!\[\]\((.*?)\)\s*((?:Figura|Figure|Fig.)\s+\d+(?:\.\d+)?\s*.*?)\n'
    
    md_text = re.sub(pattern, '', md_text)
    
    # Convert Block Formulas with Captions
    def convert_equation(match):
        equation = match.group(1).strip()
        return f" \\begin{{equation}}\n{equation}\n\\end{{equation}} "

    md_text = re.sub(r'\$\$(.*?)\$\$', convert_equation, md_text, flags=re.DOTALL)
    
    # Convert Inline Formulas
    md_text = re.sub(r'\$(.*?)\$', r'\\(\1\\)', md_text)  # Inline math: $x^2$ -> \(x^2\)
    
        
    def convert_table(match):
        table_html = match.group(2)  # Captured table HTML
        caption_before = re.search(r'^\s*(Table [IVXLCDM\d]+.*?)$', match.group(1), re.MULTILINE) if match.group(1) else None
        caption_after = re.search(r'^\s*(Table [IVXLCDM\d]+.*?)$', match.group(3), re.MULTILINE) if match.group(3) else None
        caption = caption_before.group(1) if caption_before else (caption_after.group(1) if caption_after else "Table caption")
        # Extract table rows
        rows = re.findall(r'<tr>(.*?)</tr>', table_html, re.DOTALL)
        if not rows:
            return ""  # If no rows are found, return empty string (invalid table)

        # Detect number of columns
        first_row = re.findall(r'<t[dh]>(.*?)</t[dh]>', rows[0])
        num_columns = len(first_row) if first_row else 1  # Default to 1 if no columns detected

        # Start LaTeX table
        latex_table = " \\begin{table}[h]\n\\centering\n"
        latex_table += "\\begin{tabular}{" + "|c" * num_columns + "|}\n\\hline\n"
        for row in rows:
            cells = re.findall(r'<t[dh]>(.*?)</t[dh]>', row)
            latex_table += " & ".join(cells) + " \\\\\n\\hline\n"

        latex_table += f"\\end{{tabular}}\n\\caption{{{caption}}}\n\\end{{table}}\n "
        return latex_table

    table_pattern = re.compile(
        r'\s*((?:Table|Tabla)\s+[IVXLCDM\d]+[^.\n]*[:.]?)?'  # Accepts "Table II:", "Tabla 2."
        r'\s*\n*'  # Allow one or more newlines between caption and table
        r'\s*<html><body><table>(.*?)</table></body></html>'  # detect the HTML table content
        r'\s*((?:Table|Tabla)\s+[IVXLCDM\d]+[^.\n]*[:.]?)?',  # Optional caption after
        re.DOTALL | re.IGNORECASE
    )

    # Replace Markdown tables with LaTeX
    md_text = re.sub(table_pattern, convert_table, md_text)
    
    #remove citations
    md_text = re.sub(r'\[\d+\][,\-]?', '', md_text)
    
    # Define regex pattern to match the section title in different languages
    pattern = r'(\\section\s*\{(REFERENCES|REFERENCIAS|BIBLIOGRAPHY|BIBLIOGRAFIA)\s*\}).*'

    # Remove everything after the matched section title
    md_text = re.sub(pattern, r'\1', md_text, flags=re.IGNORECASE | re.DOTALL)
    
    
    return md_text 

with open("05_SPA.md", "r", encoding="utf-8") as f:
    markdown_content = f.read()
    
latex_content = md_to_latex(markdown_content)
latex_content

' # Demostración de Procesamiento de Documentos en Español con IA\n\nimmediate  \n\nLa inteligencia artificial (IA) es un campo multidisciplinario de la informática que busca desarrollar sistemas capaces de realizar tareas que normalmente requieren inteligencia humana, como el reconocimiento de patrones, la toma de decisiones, el aprendizaje \\(\\mathbf{y}\\) la resolución de problemas. La IA se divide en varias subáreas, como el aprendizaje automático (machine learning), la visión por computadora, el procesamiento del lenguaje natural (NLP) \\(\\mathbf{y}\\) la robótica. Estos sistemas pueden ser entrenados para mejorar su desempeño con el tiempo mediante el uso de grandes cantidades de datos y algoritmos especializados.  \n\n # Introducción\n\nEl concepto de inteligencia artificial posee diversas perspectivas de acuerdo con el contexto en el que se usa. Sin embargo, una definición clásica y de amplia aceptación por la comunidad científica y tecnológica es la proporcionada por los pro

## CHUNKING RECURSIVE + LLM

In [19]:
metadata = []

def count_tokens(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return len(tokens)
    
def llm_chunk(paragraph):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    Eres un asistente experto en segmentación semántica, 
    tu tarea es dividir cualquier texto en segmentos coherentes y con significado semántico.<|start_header_id|>user<|end_header_id|>
    Por favor, divide el siguiente texto en segmentos con significado semántico. 
    Cada segmento debe contener oraciones completas y debe tener entre 380 o 400 tokens como máximo. En caso de encontrar formulas matematicas en formato latex, debes mantenerlas obligatoriamente.
    Al inicio de cada segmento debes añadir la etiqueta <chunk_begining> y al final de cada fragmento, añade <chunk_end>.
    Texto:
    
    \"\"\"{paragraph}\"\"\"
    
    <|start_header_id|>assistant<|end_header_id|>
    """
    
    generator = pipeline("text-generation", model=llm_model, tokenizer=llm_tokenizer)
    output = generator(
        prompt,
        do_sample=False,
        max_new_tokens=400,
        return_full_text=False
    )
    answer = output[0]['generated_text']
    pattern = r"<chunk_begining>\s*(.*?)\s*<chunk_end>"
    chunks = re.findall(pattern, answer, re.DOTALL)
    
    return chunks
    
def split_markdown_sections(markdown_text: str):
    """
    Splits markdown into sections every time there's a title (#) or subtitle (##),
    BUT merges a subtitle immediately following a title as one section.
    Returns list of dicts: {"section_title": ..., "content": ...}
    """
    # Find all headings and their positions
    #heading_pattern = re.compile(r'^(#{1,2}) (.+)$', re.MULTILINE)
    heading_pattern = re.compile(r'^\s*(#{1,2})\s+(.+)$', re.MULTILINE)
    matches = list(heading_pattern.finditer(markdown_text))

    sections = []
    i = 0

    while i < len(matches):
        match = matches[i]
        level = len(match.group(1))
        heading_text = match.group(2).strip()
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(markdown_text)

        section_title = f"{match.group(1)} {heading_text}"
        section_content_start = start
        section_content_end = end
        i += 1

        # If current is Title (#) and next is Subtitle (##) — merge
        if level == 1 and i < len(matches):
            next_match = matches[i]
            next_level = len(next_match.group(1))
            if next_level == 2:
                # Merge next subtitle into current title
                next_heading_text = next_match.group(2).strip()
                section_title += f"\n{next_match.group(1)} {next_heading_text}"
                section_content_start = next_match.end()
                end = matches[i + 1].start() if i + 1 < len(matches) else len(markdown_text)
                section_content_end = end
                i += 1  # Skip the subtitle match

        content = markdown_text[section_content_start:section_content_end].strip()
        sections.append({
            "section_title": section_title,
            "content": content
        })

    return sections


def recursive_chunk(text: str, max_tokens: int = MAX_TOKENS):
    """
    Recursively splits text into chunks no larger than max_tokens.
    First case scenario is when the paragraph´s token lenght is the same or smaller than max_tokens
    Second case scenario is when the paragraph´s token lenght is bigger than max_tokens. In that case we split by /n/n
    """
            
    chunks = []
    tokens = tokenizer.encode(text, add_special_tokens=False)
    
    if len(tokens) < max_tokens:
        chunks.append(text)

    else:
        section = text.split('\n\n')
        for paragraph in section:
            chunk_text =  llm_chunk(paragraph)
            chunks.append(chunk_text)

    return chunks

def retrieve_top_k(model, id_map, metadata, query, k=5):
    # Function to retrieve top 3 contexts
    query_embedding = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)  # Normalize query for cosine similarity
    start_time = time.time()
    distances, indices = id_map.search(query_embedding, k)
    end_time = time.time()
    retrieval_time = end_time - start_time
    
    results = []
    for i, idx in enumerate(indices[0]):
        if idx != -1:
            meta = metadata[idx]
            results.append({
                #"full_paragraph": meta["full_paragraph"],
                "chunk_text": meta["chunk_text"],
                "distance": float(distances[0][i]) 
            })
    
    return results, retrieval_time
    
def chunk_markdown(markdown_text: str, max_tokens: int = MAX_TOKENS):
    """
        Splits markdown into chunks
    """
    index_start_time = time.time()
    sections = split_markdown_sections(markdown_text)   
    all_chunks = []
    
    for section in sections:
        title = section['section_title']
        content = section['content']
        
        combined_text = f"{title}\n{content}".strip()
        
        llm_chunks = recursive_chunk(combined_text)

        for i in llm_chunks:
            all_chunks.append(i)
            metadata.append({
                "full_paragraph": combined_text,
                "chunk_text": i
            })

    return all_chunks

chunks = chunk_markdown(latex_content)
chunks


['# Demostración de Procesamiento de Documentos en Español con IA\nimmediate  \n\nLa inteligencia artificial (IA) es un campo multidisciplinario de la informática que busca desarrollar sistemas capaces de realizar tareas que normalmente requieren inteligencia humana, como el reconocimiento de patrones, la toma de decisiones, el aprendizaje \\(\\mathbf{y}\\) la resolución de problemas. La IA se divide en varias subáreas, como el aprendizaje automático (machine learning), la visión por computadora, el procesamiento del lenguaje natural (NLP) \\(\\mathbf{y}\\) la robótica. Estos sistemas pueden ser entrenados para mejorar su desempeño con el tiempo mediante el uso de grandes cantidades de datos y algoritmos especializados.',
 '# Introducción\nEl concepto de inteligencia artificial posee diversas perspectivas de acuerdo con el contexto en el que se usa. Sin embargo, una definición clásica y de amplia aceptación por la comunidad científica y tecnológica es la proporcionada por los profesore

In [20]:
#EMBEDDINGS FOR RETRIEVAL

index_start_time = time.time()
print("Computing embeddings...")
embeddings = model.encode(chunks, convert_to_numpy=True)
faiss.normalize_L2(embeddings)  # Normalize embeddings for cosine similarity

print("Creating FAISS index...")
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # Use Inner Product for Cosine Similarity
id_map = faiss.IndexIDMap(index)

print("Storing embeddings...") # Assign unique IDs and store embeddings
ids = np.arange(len(embeddings))
id_map.add_with_ids(embeddings, ids)

print("Saving FAISS index...")
faiss.write_index(id_map, "rag_faiss.index")

index_end_time = time.time()
indexing_time = index_end_time - index_start_time
print(f"Indexing complete in {indexing_time:.4f} seconds! Stored FAISS index and metadata.")



Computing embeddings...
Creating FAISS index...
Storing embeddings...
Saving FAISS index...
Indexing complete in 0.0706 seconds! Stored FAISS index and metadata.


In [21]:
# 
all_results = []
question = "Qué es la inteligencia artificial?"

contexts, retrieval_time = retrieve_top_k(model, id_map, metadata, question)

all_results.append({
        "question": question,
        "contexts": contexts,
        #"retrieval_time": float(retrieval_time)  # Convert to regular float
    })

all_results

[{'question': 'Qué es la inteligencia artificial?',
  'contexts': [{'chunk_text': '# Demostración de Procesamiento de Documentos en Español con IA\nimmediate  \n\nLa inteligencia artificial (IA) es un campo multidisciplinario de la informática que busca desarrollar sistemas capaces de realizar tareas que normalmente requieren inteligencia humana, como el reconocimiento de patrones, la toma de decisiones, el aprendizaje \\(\\mathbf{y}\\) la resolución de problemas. La IA se divide en varias subáreas, como el aprendizaje automático (machine learning), la visión por computadora, el procesamiento del lenguaje natural (NLP) \\(\\mathbf{y}\\) la robótica. Estos sistemas pueden ser entrenados para mejorar su desempeño con el tiempo mediante el uso de grandes cantidades de datos y algoritmos especializados.',
    'distance': 0.8579941987991333},
   {'chunk_text': '# Introducción\nEl concepto de inteligencia artificial posee diversas perspectivas de acuerdo con el contexto en el que se usa. Sin